# Pruebas básicas
Una vez vistas las clases por las que está formada gymnasium y de que trata, se van a realizar varias pruebas con un entorno, en concreto CartPole para ver el funcionamiento general de la librería y sus posibilidades. <br>
<img src="./media/cart_pole.gif" width="350px"/>
El entorno *CartPole*, cumple las siguientes características:
- **Espacio de acciones** 
    0. para mover el carro a la izquierda
    1. para mover el carro a la derecha
- **Espacio de observaciones**:
    0. Posición del carro
    1. Velocidad del carro
    2. Ángulo del palo
    3. Velocidad angular del palo
    
La misión del entorno, es mantener el palo encima del carro el máximo tiempo posible.

En esta primera prueba, las acciones que se van a tomar van a ser random, es decir, no se va a seguir ninguna política ni ningún tipo de entrenamiento. Este ejemplo, es únicamente para ver como sería una salida típica.

In [2]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() #primer reset necesario

total = 0

for episode in range(20):
    observation, info = env.reset()
    for t in range(1000): #tiempo que está en ejecución el entorno
        action = env.action_space.sample() #accion random
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated: #finalizada la ejecución del anterior
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()

Episodio terminado después de 20 timesteps
Episodio terminado después de 21 timesteps
Episodio terminado después de 14 timesteps
Episodio terminado después de 23 timesteps
Episodio terminado después de 18 timesteps
Episodio terminado después de 10 timesteps
Episodio terminado después de 16 timesteps
Episodio terminado después de 18 timesteps
Episodio terminado después de 20 timesteps
Episodio terminado después de 13 timesteps
Episodio terminado después de 16 timesteps
Episodio terminado después de 11 timesteps
Episodio terminado después de 25 timesteps
Episodio terminado después de 11 timesteps
Episodio terminado después de 36 timesteps
Episodio terminado después de 22 timesteps
Episodio terminado después de 13 timesteps
Episodio terminado después de 32 timesteps
Episodio terminado después de 32 timesteps
Episodio terminado después de 15 timesteps
Tiempo medio: 19.3


Una posible política para este problema sería que el palo intentase estar centrado, ya que la meta es mantenerse el máximo tiempo posible. Para ello, si el palo es mayor que 0, se estará cayendo a la derecha por lo que habrá que mover el carro a la derecha también. Un ejemplo de implementación sería:

In [3]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() 

total = 0

for episode in range(20):
    observation, info = env.reset()
    for t in range(1000): 
        if observation[2] > 0.0:
            observation, reward, terminated, truncated, info = env.step(1)
        else:
            observation, reward, terminated, truncated, info = env.step(0)

        if terminated or truncated: 
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()





Episodio terminado después de 38 timesteps
Episodio terminado después de 55 timesteps
Episodio terminado después de 46 timesteps
Episodio terminado después de 41 timesteps
Episodio terminado después de 49 timesteps
Episodio terminado después de 40 timesteps
Episodio terminado después de 40 timesteps
Episodio terminado después de 52 timesteps
Episodio terminado después de 40 timesteps
Episodio terminado después de 38 timesteps
Episodio terminado después de 31 timesteps
Episodio terminado después de 40 timesteps
Episodio terminado después de 46 timesteps
Episodio terminado después de 49 timesteps
Episodio terminado después de 39 timesteps
Episodio terminado después de 38 timesteps
Episodio terminado después de 39 timesteps
Episodio terminado después de 25 timesteps
Episodio terminado después de 48 timesteps
Episodio terminado después de 42 timesteps
Tiempo medio: 41.8


Después de esta ejecución, se puede comprobar que se duplica el tiempo medio que aguanta el palo, por lo que la política escogida sería correcta. <br> <br>

Otra forma de posible, sería que además del ángulo, también la velocidad angular (*observation[3]*) sea positiva, ya que en casos en los que este prácticamente equilibrado, se haría un movimiento que podría ser innecesario. Un ejemplo de esta política sería:

In [2]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() 

total = 0

for episode in range(20):
    observation, info = env.reset()
    for t in range(1000): 
        if observation[2] > 0.0 and observation[3] > 0.0:
            observation, reward, terminated, truncated, info = env.step(1)
        else:
            observation, reward, terminated, truncated, info = env.step(0)

        if terminated or truncated: 
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()


Episodio terminado después de 135 timesteps
Episodio terminado después de 117 timesteps
Episodio terminado después de 196 timesteps
Episodio terminado después de 142 timesteps
Episodio terminado después de 180 timesteps
Episodio terminado después de 135 timesteps
Episodio terminado después de 181 timesteps
Episodio terminado después de 118 timesteps
Episodio terminado después de 165 timesteps
Episodio terminado después de 220 timesteps
Episodio terminado después de 168 timesteps
Episodio terminado después de 131 timesteps
Episodio terminado después de 216 timesteps
Episodio terminado después de 177 timesteps
Episodio terminado después de 123 timesteps
Episodio terminado después de 191 timesteps
Episodio terminado después de 196 timesteps
Episodio terminado después de 134 timesteps
Episodio terminado después de 156 timesteps
Episodio terminado después de 229 timesteps
Tiempo medio: 165.5


In [4]:
env.close()

Para aplicar Q-learning a este problema continuo, sería neceario discretizarlo para poder formar la tabla o utilizar Deep Q-learning. Para demostrar un caso de uso de Aprendizaje Reforzado combinado con la librería (uso común de la misma), se va a aplicar y así de esta misma forma, se puede comparar el resultado con el obtenido por las políticas anteriores.<br> <br>

Las características del entorno, serían las comentadas anteriormente.

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import random
import numpy as np

class DQN():
    def __init__(self, env):
        self.max_episodes = 20000
        self.max_actions = 99
        
        self.epsilon = 1.0 
        self.max_epsilon = 1.0
        self.min_epsilon = 0.01 
        self.decay_rate = 0.005
        self.learning_rate = 0.001
        
        
        self.input_n = env.observation_space.shape[0]
        self.output_n = env.action_space.n
        
        self.nn = self.nn_model(env)
        self.env = env
        
        print(self.output_n)
        
    def nn_model(self, env):
        model = Sequential()
        model.add(Dense(24, input_dim=self.input_n, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, input_dim=self.output_n, activation='linear'))
        
        model.compile(loss='mse',
                      optimizer=Adam(learning_rate=self.learning_rate))
        return model
    
    def act(self, observation):
        if np.random.rand() <= self.epsilon:
            action = self.env.action_space.sample()
        else:
            action = np.argmax(self.nn.predict(observation)[0]) 
            print(action)
        #Se reduce epsilon
        if self.epsilon > self.min_epsilon:
            self.epsilon *= self.decay_rate
        return action
        
    def train(self, episodes):
        replay_buffer = []
        reward_buffer = []
        episode_reward = 0.0
        for episode in range(episodes):
            observation, info = env.reset()
            for s in range(10000): 

                action = self.act(observation)
                new_observation, reward, terminated, truncated, info = env.step(action)
                transition = (observation, action, reward, terminated, new_observation)
                replay_buffer.append(transition)
                observation = new_observation

                episode_reward += reward
                

                if s % 100 == 0:
                    print('Step: {}'.format(s))
                    print('Avg reward: {}'.format(np.mean(reward_buffer)))

                if terminated or truncated: 
                    observation, info = env.reset()
                    reward_buffer.append(episode_reward)
                    episode_reward = 0
                    break
    

    def test(self, Q):
        pass

In [1]:
"""
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
d = DQN(env)
d.train(1000)
"""

'\nimport gymnasium as gym\nenv = gym.make("CartPole-v1", render_mode="human")\nd = DQN(env)\nd.train(1000)\n'

In [6]:
env.close()

In [13]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() 

total = 0

d = DQN(env)

for episode in range(2):
    observation, info = env.reset()
    for t in range(1000): 
        action = d.act(observation)
            
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated: 
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
     
            
print("Tiempo medio: {}".format(total/20))
env.close()

2


ValueError: in user code:

    File "/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/alberto/anaconda3/envs/gymnasium

In [14]:
env.close()